**Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones**

**Exploración y Curación de Datos**

*Edición 2023*

### Trabajo práctico entregable - Grupo 22 - Parte 1

**Integrantes:**
- Chevallier-Boutell, Ignacio José
- Ribetto, Federico Daniel
- Rosa, Santiago
- Spano, Marcelo

**Seguimiento:** Meinardi, Vanesa


# Trabajo práctico entregable - parte 1

En esta notebook, vamos a cargar el conjunto de datos de [la compentencia Kaggle](https://www.kaggle.com/dansbecker/melbourne-housing-snapshot) sobre estimación de precios de ventas de propiedades en Melbourne, Australia.

Utilizaremos el conjunto de datos reducido producido por [DanB](https://www.kaggle.com/dansbecker). Hemos subido una copia a un servidor de la Universidad Nacional de Córdoba para facilitar su acceso remoto.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import seaborn as sns
sns.set_context('talk')

from sqlalchemy import create_engine, text

In [ ]:
import plotly
plotly.__version__


## Ejercicio 1 SQL: 

1. Crear una base de datos en SQLite utilizando la libreria [SQLalchemy](https://stackoverflow.com/questions/2268050/execute-sql-from-file-in-sqlalchemy).
https://docs.sqlalchemy.org/en/14/core/engines.html#sqlite

2. Ingestar los datos provistos en 'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/melb_data.csv' en una tabla y el dataset generado en clase con datos de airbnb y sus precios por codigo postal en otra.

3. Implementar consultas en SQL que respondan con la siguiente información:

    - cantidad de registros totales por ciudad.
    - cantidad de registros totales por barrio y ciudad.

4. Combinar los datasets de ambas tablas ingestadas utilizando el comando JOIN de SQL  para obtener un resultado similar a lo realizado con Pandas en clase.  



### Actividad 1
#### Conexión

Se crea una conexión a una base de datos SQLite

In [ ]:
engine = create_engine('sqlite:///database.sqlite3', echo=True)

### Actividad 2
#### Ingesta de datos de melb_data

Se leen los datos de melb_data utilizando pandas:

In [ ]:
melb_df = pd.read_csv(
    'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/melb_data.csv')
melb_df[:3]

Se guardan los datos de melb_data en la tabla "prop_melb" en la base de datos creada previamente.

In [ ]:
melb_df.to_sql('prop_melb', con=engine, if_exists="replace")

#### Ingesta de datos de airbnb

Se leen los datos crudos de airbnb

In [ ]:
# data source:
# https://www.kaggle.com/tylerx/melbourne-airbnb-open-data?select=cleansed_listings_dec18.csv
interesting_cols = [
  'description', 'neighborhood_overview',
  'street', 'neighborhood', 'city', 'suburb', 'state', 'zipcode',
  'price', 'weekly_price', 'monthly_price',
  'latitude', 'longitude',
]
airbnb_df = pd.read_csv(
    'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/cleansed_listings_dec18.csv',
    usecols=interesting_cols)

Queremos agregar información al dataset de melb_data utilizando el código postal como clave para joinear. Por eso debemos asegurarnos que se encuentre limpia.

In [ ]:
# Vemos que la variable zipcode tiene diferentes tipos de datos:
airbnb_df['zipcode'].value_counts().head(5)

In [ ]:
# Se estandariza el tipo de datos para la columna zipcode
airbnb_df['zipcode'] = pd.to_numeric(airbnb_df.zipcode, errors='coerce')

In [ ]:
airbnb_df.zipcode.value_counts().head(5)

Se guardan los datos de airbnb en la tabla "airbnb" en la base de datos creada previamente.

In [ ]:
airbnb_df.to_sql('airbnb', con=engine, if_exists="replace")

### Actividad 3
Implementar consultas en SQL que respondan con la siguiente información:

- cantidad de registros totales por ciudad.
- cantidad de registros totales por barrio y ciudad.

Se define una función para ejecutar queries con la conexión creada previamente:

In [ ]:
def execute_query(query):
  with engine.connect() as con:
    rs = con.execute(text(query))
    df_rs = pd.DataFrame(rs.fetchall())
  return df_rs

Vemos la cantidad de registros totales por ciudad agrupando por la columna CITY:

In [ ]:
query_city = """SELECT CITY AS CIUDAD, COUNT(*) AS CANT_REGISTROS 
                FROM AIRBNB
                GROUP BY CITY"""
df_city = execute_query(query_city)
df_city.head(10)

Vemos la cantidad de registros totales por barrio y ciudad agrupando por las columnas NEIGHBORHOOD y CITY:

In [ ]:
query_neighborhood_city = """SELECT NEIGHBORHOOD AS BARRIO, CITY AS CIUDAD, COUNT(*) AS CANT_REGISTROS 
                              FROM AIRBNB
                              GROUP BY NEIGHBORHOOD, CITY"""
df_neighborhood_city = execute_query(query_neighborhood_city)
df_neighborhood_city.tail(10)

### Actividad 4
Combinar los datasets de ambas tablas ingestadas utilizando el comando JOIN de SQL  para obtener un resultado similar a lo realizado con Pandas en clase.

In [ ]:
# Se realiza una subquery en la que se define AIRBNB_AGG con las agregaciones generadas en clase con Pandas.
# AIRBNB_AGG es joineada con la tabla original PROP_MELB
query_join = """WITH AIRBNB_AGG AS (
                  SELECT ZIPCODE, 
                    AVG(PRICE) AS AIRBNB_PRICE_MEAN,
                    COUNT(PRICE) AS AIRBNB_RECORD_COUNT,
                    AVG(WEEKLY_PRICE) AS AIRBNB_WEEKLY_PRICE_MEAN,
                    AVG(MONTHLY_PRICE) AS AIRBNB_MONTHLY_PRICE_MEAN
                  FROM AIRBNB
                  GROUP BY ZIPCODE
                )
                SELECT * FROM PROP_MELB A
                LEFT JOIN AIRBNB_AGG B 
                ON A.Postcode = B.zipcode"""
df_join = execute_query(query_join)

In [ ]:
df_join.sample(10)

## Ejercicio 2 - Pandas: 

1. Seleccionar un subconjunto de columnas que les parezcan relevantes al problema de predicción del valor de la propiedad. Justificar las columnas seleccionadas y las que no lo fueron.
 - Eliminar los valores extremos que no sean relevantes para la predicción de valores de las propiedades.

 
2. Agregar información adicional respectiva al entorno de una propiedad a partir del [conjunto de datos de AirBnB](https://www.kaggle.com/tylerx/melbourne-airbnb-open-data?select=cleansed_listings_dec18.csv) utilizado en el práctico. 
  1. Seleccionar qué variables agregar y qué combinaciones aplicar a cada una. Por ejemplo, pueden utilizar solo la columna `price`, o aplicar múltiples transformaciones como la mediana o el mínimo.
  1. Utilizar la variable zipcode para unir los conjuntos de datos. Sólo incluir los zipcodes que tengan una cantidad mínima de registros (a elección) como para que la información agregada sea relevante.
  2. Investigar al menos otras 2 variables que puedan servir para combinar los datos, y justificar si serían adecuadas o no. Pueden asumir que cuentan con la ayuda de anotadores expertos para encontrar equivalencias entre barrios o direcciones, o que cuentan con algoritmos para encontrar las n ubicaciones más cercanas a una propiedad a partir de sus coordenadas geográficas. **NO** es necesario que realicen la implementación.

Pueden leer otras columnas del conjunto de AirBnB además de las que están en `interesting_cols`, si les parecen relevantes.

### Actividad 1

Leemos el dataset nuevamente con Pandas

In [ ]:
df = pd.read_csv(
    'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/melb_data.csv')
df[:3]

Separamos las columnas categóricas de las numéricas

In [ ]:
cols = df.columns
num_cols = ['Rooms', 'Price', 'Distance', 'Bedroom2', 'Bathroom',
       'Car', 'Landsize', 'BuildingArea', 'YearBuilt',
       'Lattitude', 'Longtitude', 'Propertycount']
cat_cols = [x for x in cols if x not in num_cols and x != 'index']

#### Análisis de variables numéricas

Vemos la correlación de las variables numéricas con el precio

In [ ]:
df[num_cols].corr()['Price']

Se realiza un scatterplot del Precio en función de todas las variables numéricas

In [ ]:
for col in num_cols:
  plt.figure(figsize=(8,3))
  plt.scatter(df[col], df['Price'])
  plt.axvline(df[col].mode()[0], color='r')
  plt.grid()
  plt.title(col)
  plt.show()

Outliers:
- En la variable Bedroom2 vemos que tiene un valor extremo = 20.
- En la variable Landsize vemos que tiene un valor extremo > 400000
- En la variable BuildingArea vemos que tiene un valor extremo > 40000
- En la variable YearBuilt vemos que tiene un valor extremo en 1200

Quitamos estos valores extremos

In [ ]:
df = df[df['Bedroom2'] < 20]
df = df[df['Landsize'] < 400000]
df = df[df['BuildingArea'] < 40000]
df = df[df['YearBuilt'] > 1200]

Veamos nuevamente la correlación

In [ ]:
corr = df_join[num_cols].corr()['Price']
corr

Realizamos boxplots para las variables discretas

In [ ]:
discrete_cols = ['Rooms', 'Bedroom2', 'Bathroom', 'Car']
for col in discrete_cols:
  df_var = df[[col, 'Price']]
  df_var['count'] = df_var[col].astype(str)
  plt.figure(figsize=(8,3))
  sns.boxplot(data=df_var, x='Price', y='count')
  plt.title(col)
  plt.show()

Separamos a las variables continuas en 6 cuantiles y realizamos los boxplots del Precio.

Se eligen 6 ya que para más cuantiles el código da el siguiente error:

"ValueError: Bin labels must be one fewer than the number of bin edges"

In [ ]:
continuous_cols = [x for x in num_cols if x not in discrete_cols and x!='Price']
for col in continuous_cols:
  df_var = df[[col, 'Price']]
  df_var['quantile'] = pd.qcut(df_var[col], 6, labels=['1','2','3','4','5','6'])
  plt.figure(figsize=(8,3))
  sns.boxplot(data=df_var, x='Price', y='quantile')
  plt.title(col)
  plt.show()

Viendo la correlación y los gráficos anteriores tomamos deciciones respecto a cada columna:


*   Rooms: La correlación con el Precio es de 0.52 por lo que consideramos que es importante para la predicción del valor de la propiedad.
*   Bedroom2: La correlación con el Precio es de 0.5 por lo que consideramos que es importante para la predicción del valor de la propiedad.
*   Bathroom: La correlación con el Precio es de 0.49 por lo que consideramos que es importante para la predicción del valor de la propiedad.
*   Car: La correlación Con el Precio es de 0.25. Viendo el scatterplot y el boxplot de la variable Car no se ve una relación clara con el Precio por lo que la descartamos.
*   Distance: Si bien la correlación con el Precio es relativamente baja (-0.16), se puede observar que para valores en el scatterplot que para valores menores a 20, el precio alcanza los valores más altos mientras que para valores mayores a 20 el precio permanece acotado a valores pequeños. Por esta razón la consideramos en el dataset.
*   Landsize: Se puede observar en el boxplot que las distribuciones son diferentes para cada cuantil, por lo que consideramos que es una variable importante para predecir el valor de la propiedad y la consideramos en el dataset.
*   BuildingArea: Sucede lo mismo que en la variable Landsize, además que posee una correlación alta con el Precio, por lo que la consideramos en el dataset. 
*   YearBuilt: En el boxplot se puede observar que para cuantiles más bajos (1 y 2) el Precio alcanza valores mayores que para el resto de los cuantiles. Además la correlación es negativa, lo que tiene sentido suponiendo que, cuando más vieja es la propiedad, esta pierde valor. La consideramos en el dataset.
*   Lattitude: En el boxplot se puede observar que para cuantiles intermedios (2 y 3) el Precio alcanza valores mayores que para el resto de los cuantiles. La consideramos en el dataset.
*   Longitude: En el boxplot se puede observar que para cuantiles intermedios (4 y 5) el Precio alcanza valores mayores que para el resto de los cuantiles. La consideramos en el dataset.
*   Propertycount: La correlación Con el Precio es de -0.05. Viendo el scatterplot y el boxplot de la variable Propertycount no se ve una relación clara con el Precio por lo que la descartamos.



In [ ]:
selected_num_cols = ['Rooms', 'Bedroom2', 'Bathroom', 'Distance', 'Landsize',
                    'BuildingArea', 'YearBuilt', 'Lattitude', 'Longtitude']

#### Análisis de variables categóricas

In [ ]:
cat_cols

Veamos la cardinalidad de cada una y el porcentaje de repeticiones que tienen las 10 categorías con mayor cantidad:

In [ ]:
for col in cat_cols:
  print(col)
  print('Cardinalidad:', df[col].nunique())
  print(100*df[col].value_counts(normalize=True).iloc[:10])
  print()

* Suburb: Tiene 300 valores diferentes y la categoría que posee la mayor cantidad de casos solo tiene el 2,5% por lo que la descartamos.
* Address: La descartamos ya que la dirección es única para cada propiedad y no es representativa.
* Type: Tiene 3 valores únicos, la consideramos en el dataset.
* Method: Posee 5 valores únicos, la consideramos en el dataset.
* SellerG: Posee 214 valores únicos, pero vemos que algunos de estos valores poseen un gran porcentaje, como Nelson que tiene el 12% o Jellis con el 10%. La consideramos en el dataset.
* Date: No consieramos que sea relevante, la descartamos.
* Postcode: Tiene 190 valores diferentes y la categoría que posee la mayor cantidad de casos solo tiene el 2,5% por lo que la descartamos.
* CouncilArea: Posee 31 valores únicos, pero vemos que algunos de estos valores poseen un gran porcentaje de casos, como Moreland que tiene el 10% o Boroondara con el 9%. La consideramos en el dataset.
* Regionname: Tiene 8 valores únicos, la consideramos en el dataset.

In [ ]:
preselected_cat_cols = ['Type', 'Method', 'SellerG', 'CouncilArea', 'Regionname']

En el caso de las variables de mayor cardinalidad, CouncilArea y SellerG, creamos una categoría nueva "Other" para agrupar a los valores que posean menos del 1% de los casos.

In [ ]:
for col in ['SellerG', 'CouncilArea']:
  value_counts = 100*df[col].value_counts(normalize=True)
  lower_values = value_counts[value_counts<1].index.tolist()
  df[col] = df[col].apply(lambda x: 'Other' if x in lower_values else x)

Veamos la nueva cardinalidad de ambas variables

In [ ]:
df['SellerG'].nunique()

In [ ]:
df['CouncilArea'].nunique()

Ahora veamos los boxplots para las variables categóricas preseleccionadas

In [ ]:
for col in preselected_cat_cols:
  values = df[col].value_counts(normalize=True).index[:10].tolist()
  plt.figure(figsize=(8,4))
  sns.boxplot(data=df[df[col].isin(values)], y=col, x='Price')
  plt.show()

En todos los casos vemos que hay valores para los cuales el precio tiende a tener precios mayores que en otros por lo que seleccionamos todas estas variables

In [ ]:
selected_cat_cols = preselected_cat_cols

#### Unificamos variables seleccionadas numéricas y categóricas

In [ ]:
selected_cols = selected_num_cols + selected_cat_cols

In [ ]:
selected_cols

In [ ]:
df = df[selected_cols + ['Price', 'Postcode']]

In [ ]:
df.head()

### Actividad 2

In [ ]:
airbnb_df = pd.read_csv(
    'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/cleansed_listings_dec18.csv')

In [ ]:
# Se estandariza el tipo de datos para la columna zipcode
airbnb_df['zipcode'] = pd.to_numeric(airbnb_df.zipcode, errors='coerce')

In [ ]:
airbnb_df.head(3)

In [ ]:
sns.scatterplot(airbnb_df.zipcode.value_counts().values)
plt.axhline(100, color='g')
plt.axhline(50, color='r')

Incluimos los zipcodes que tengan una cantidad mayor o igual a 100 registros para que la información agregada sea relevante

In [ ]:
value_counts = airbnb_df.zipcode.value_counts()
value_counts = value_counts[value_counts>=100]
airbnb_df = airbnb_df[airbnb_df['zipcode'].isin(value_counts.index.tolist())]

In [ ]:
airbnb_price_by_zipcode = airbnb_df.groupby('zipcode')\
  .agg({'price': ['mean', 'count'], 'weekly_price': 'mean',
        'monthly_price': 'mean', 'number_of_reviews': ['sum', 'mean'],
        'review_scores_rating': ['min', 'max', 'mean']})\
  .reset_index()
# Flatten the two level columns
airbnb_price_by_zipcode.columns = [
  ' '.join(col).strip()
  for col in airbnb_price_by_zipcode.columns.values]

In [ ]:
airbnb_price_by_zipcode.columns

In [ ]:
# Rename columns
airbnb_price_by_zipcode = airbnb_price_by_zipcode.rename(
    columns={'price mean': 'airbnb_price_mean',
             'price count': 'airbnb_record_count',
             'weekly_price mean': 'airbnb_weekly_price_mean',
             'monthly_price mean': 'airbnb_monthly_price_mean',
             'number_of_reviews sum': 'airbnb_number_of_reviews_sum', 
             'number_of_reviews mean': 'airbnb_number_of_reviews_mean', 
             'review_scores_rating min': 'airbnb_review_scores_rating_min', 
             'review_scores_rating max': 'airbnb_review_scores_rating_max',
             'review_scores_rating mean': 'airbnb_review_scores_rating_mean'}
)

In [ ]:
airbnb_price_by_zipcode.head()

In [ ]:
df_join = df.merge(
    airbnb_price_by_zipcode, how='left',
    left_on='Postcode', right_on='zipcode'
)
df_join.sample(5)

### Actividad 3

Se podría utilizar la variable Suburb que se encuentra en ambos datasets y posee una cardinalidad mayor al código postal en ambos casos

In [ ]:
print('Cardinalidad Código postal:', melb_df.Postcode.nunique())
print('Cardinalidad Suburbio:', melb_df.Suburb.nunique())

In [ ]:
print('Cardinalidad Código postal:', airbnb_df.zipcode.nunique())
print('Cardinalidad Suburbio:', airbnb_df.suburb.nunique())

Otra opción sería utilizar la latitud y longitud que se encuentra en ambos datasets y encontrar todas las propiedades que se encuentren a cierta distancia.

## Ejercicio 3:

Crear y guardar un nuevo conjunto de datos con todas las transformaciones realizadas anteriormente.

In [ ]:
df_join.head()

Guardamos el dataset en un archivo csv:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/new_dataset.csv'

df_join.to_csv(path)

## Ejercicios opcionales:

1. Armar un script en python (archivo .py) [ETL](https://towardsdatascience.com/what-to-log-from-python-etl-pipelines-9e0cfe29950e) que corra los pasos de extraccion, transformacion y carga, armando una funcion para cada etapa del proceso y luego un main que corra todos los pasos requeridos.

2. Armar un DAG en Apache Airflow que corra el ETL. (https://airflow.apache.org/docs/apache-airflow/stable/tutorial.html)